<a href="https://colab.research.google.com/github/yohanesnuwara/mem/blob/master/isotropy_anisotropy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np

# Isotropic

In [0]:
Vp = 2000 # m/s
Vs = 1000
rho = 2000 # kg/m3

C33 = (rho * (Vp)**2) * (1E-09) # GPa
C44 = (rho * (Vs)**2) * (1E-09) # GPa

C11 = C33
C12 = C33 - 2 * C44
C13 = C33 - 2 * C44
C22 = C33
C23 = C33 - 2 * C44
C55 = C66 = C44
C_matrix = np.array([[C11,  C12,  C13,    0,    0,    0],
                     [0,    C22,  C23,    0,    0,    0],
                     [0,    0,    C33,    0,    0,    0],
                     [0,    0,      0,  C44,    0,    0],
                     [0,    0,      0,    0,  C55,    0],
                     [0,    0,      0,    0,    0,  C66]])
C_inv = np.linalg.inv(C_matrix)
print('C33:', C33, 'GPa, and C44:', C44, 'GPa \n')
print('Isotropic elastic stiffness tensor:')
print(C_matrix, '\n')
print('Compliance tensor (in GPa):')
print(C_inv)

C33: 8.0 GPa, and C44: 2.0 GPa 

Isotropic elastic stiffness tensor:
[[8. 4. 4. 0. 0. 0.]
 [0. 8. 4. 0. 0. 0.]
 [0. 0. 8. 0. 0. 0.]
 [0. 0. 0. 2. 0. 0.]
 [0. 0. 0. 0. 2. 0.]
 [0. 0. 0. 0. 0. 2.]] 

Compliance tensor (in GPa):
[[ 0.125   -0.0625  -0.03125  0.       0.       0.     ]
 [ 0.       0.125   -0.0625   0.       0.       0.     ]
 [ 0.       0.       0.125    0.       0.       0.     ]
 [ 0.       0.       0.       0.5      0.       0.     ]
 [ 0.       0.       0.       0.       0.5      0.     ]
 [ 0.       0.       0.       0.       0.       0.5    ]]


In [0]:
sigma_xx = 5000
sigma_yy = 15000
sigma_zz = 10000
stress_matrix = np.array([[sigma_xx],
                          [sigma_yy],
                          [sigma_zz],
                          [0],
                          [0],
                          [0]])
strain_matrix = np.dot((C_inv / 145000), stress_matrix)
strain_matrix

array([[-0.00431034],
       [ 0.00862069],
       [ 0.00862069],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ]])

# Anisotropy Type VTI (Vertical Transverse Isotropy)

£££

## Hudson (1981) and Cheng (1993) Crack Model

In [0]:
Vp = 2000 # m/s
Vs = 1000
rho = 2000 # kg/m3
poro = 0.15
alpha = 0.01
Kf = 2.24 # GPa
Gf = 0

C33_eff = (rho * (Vp)**2) * (1E-09) # GPa
C44_eff = (rho * (Vs)**2) * (1E-09) # GPa
print('Effective C33:', C33_eff, 'GPa, and C44:', C44_eff, 'GPa')

Effective C33: 8.0 GPa, and C44: 2.0 GPa


In [0]:
from scipy.optimize import fsolve

def f(y):
  K0, G0 = y

  # Hashin-Shtrikman
  # K_HS = K0 + (poro / (((Kf - K0)**(-1)) + ((1 - poro) * ((K0 + ((4 / 3) * G0))**(-1)))))
  # G_HS = G0 + (poro / (((Gf - G0)**(-1)) + ((2 * (1 - poro)) * ((K0 + 2 * G0) / (5 * G0 * (K0 + ((4 / 3) * G0)))))))

  K_HS = K0 + (poro / ((1 / (Kf - K0)) + ((1 - poro) * ((K0 + ((4 / 3) * G0))**(-1)))))
  G_HS = G0 + (poro / ((1 / (Gf - G0)) + ((2 * (1 - poro)) * ((K0 + 2 * G0) / (5 * G0 * (K0 + ((4 / 3) * G0)))))))

  lame_lambda = K_HS - ((2 / 3) * G_HS)
  lame_mu = G_HS

  crack_dens = (3 * poro) / (4 * np.pi * alpha)
  K = (Kf * (lame_lambda + 2 * lame_mu)) / (np.pi * alpha * (lame_lambda + lame_mu)) 

  # stiffness 33
  U3 = ((4 * (lame_lambda + 2 * lame_mu)) / (3 * (lame_lambda + lame_mu))) * (1 / (1 + K))
  C33_0 = lame_lambda + 2 * lame_mu
  C33_1 = -(((lame_lambda + 2 * lame_mu)**2) / lame_mu) * crack_dens * U3

  # stiffness 44
  U1 = ((16 * (lame_lambda + 2 * lame_mu)) / (3 * (3 * lame_lambda + 4 * lame_mu)))
  C44_0 = lame_mu
  C44_1 = -lame_mu * crack_dens * U1

  # minimized function
  f1 = C33_0 + C33_1 - C33_eff
  f2 = C44_0 + C44_1 - C44_eff

  return[f1, f2]

solve = fsolve(f, [1, 1]) # initial guess
solve

/usr/local/lib/python3.6/dist-packages/scipy/optimize/minpack.py:162: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


array([2.79067876, 0.52849871])

In [0]:
K0 = 5
G0 = 2
alpha = 0.05

A = 1 / (Kf - K0)
B = 1 / (Gf - G0)
C = 1 - poro
D = 1 / (K0 + ((4/3) * G0))
E = (K0 + 2 * G0) / (5 * G0 * (K0 + ((4/3) * G0)))

K_HS = K0 + (poro / (A + C * D))
G_HS = G0 + (poro / (B + (2 * C) * E))

lame_lambda = K_HS - ((2 / 3) * G_HS)
lame_mu = G_HS

crack_dens = (3 * poro) / (4 * np.pi * alpha)
K = (Kf * (lame_lambda + 2 * lame_mu)) / (np.pi * alpha * (lame_lambda + lame_mu)) 

# stiffness 33
U3 = ((4 * (lame_lambda + 2 * lame_mu)) / (3 * (lame_lambda + lame_mu))) * (1 / (1 + K))
C33_0 = lame_lambda + 2 * lame_mu
C33_1 = -(((lame_lambda + 2 * lame_mu)**2) / lame_mu) * crack_dens * U3

# stiffness 44
U1 = (16 * (lame_lambda + (2 * lame_mu))) / (3 * ((3 * lame_lambda) + (4 * lame_mu)))
C44_0 = lame_mu
C44_1 = -(lame_mu) * crack_dens * U1

C33_eff = C33_0 + C33_1
C44_eff = C44_0 + C44_1
C33_eff, C44_eff

(4.667468049182462, -0.7638178328023273)

In [0]:
lame_lambda, lame_mu, U1

(3.4029758205888525, 1.5007235890014472, 2.10691451232897)

In [0]:
# second order correction
q = 15 * (lame_lambda**2 / lame_mu**2) + 15 * (lame_lambda / lame_mu) + 28

C33_2 = (q * (lame_lambda + 2 * lame_mu) * ((crack_dens * U3)**2)) / 15
C44_2 = (2 * lame_mu * (3 * lame_lambda + 8 * lame_mu) * ((crack_dens * U1)**2)) / 15

# effective moduli with Cheng's (1993) Pade approximation
def Pade(Cij_0, Cij_1, Cij_2):
  b = Cij_2 / (Cij_1 * crack_dens)
  a = (Cij_1 / (Cij_0 * crack_dens)) - 1
  Cij_eff = Cij_0 * ((1 - a * crack_dens) / (1 + b * crack_dens))
  return(Cij_eff)

C33_eff = Pade(C33_0, C33_1, C33_2)
C44_eff = Pade(C44_0, C44_1, C44_2)
C33_eff, C44_eff 

(14.768262774063851, -1.3950359797123446)

In [0]:
K_HS, G_HS

(3.685607349438585, 2.934010152284264)

In [0]:
(3 * 0.15) / (4 * 3.14 * 0.01)

3.5828025477707

In [0]:
U1

2.0873479686229217

In [0]:
a = (-2)**(-1)
a

-0.5

## ***

Background stiffness moduli ($C_{ij}^0$):

$$C_{11}^0=C_{33}^0=\lambda+2\mu=\rho V_p^2$$

$$C_{44}^0=C_{66}^0=\mu=\rho V_s^2$$

$$C_{13}^0 = C_{11}^0 - 2 C_{44}^0=\lambda$$ 




In [0]:
Vp = 2000 # m/s
Vs = 1000
rho = 2000 # kg/m3

# background moduli
C11_b = (rho * (Vp)**2) * (1E-09) # GPa
C33_b = C11_b
C44_b = (rho * (Vs)**2) * (1E-09) # GPa
C66_b = C44_b
C13_b = C11_b - 2 * C44_b

print('Background moduli C11:', C11_b, 'GPa, C13:', C13_b, 'GPa, C33:', C33_b, 'GPa, C44:', C44_b, 'GPa, and C66:', C66_b, 'GPa')

Background moduli C11: 8.0 GPa, C13: 4.0 GPa, C33: 8.0 GPa, C44: 2.0 GPa, and C66: 2.0 GPa


First order correction to stiffness moduli ($C_{ij}^1$):

$$C_{11}^1=-\frac{\lambda^2}{\mu} \epsilon U_3$$

$$C_{13}^1=-\frac{\lambda(\lambda+2\mu)^2}{\mu} \epsilon U_3$$

$$C_{33}^1=-\frac{(\lambda+2\mu)^2}{\mu} \epsilon U_3$$

$$C_{44}^1=-\mu \epsilon U_1$$

$$C_{66}^1=0$$

Where $\epsilon$ is the crack density: $\epsilon=\frac{3 \phi}{4 \pi \alpha}$, $\alpha$ is aspect ratio of crack ($\alpha<0.12$)

And $U$ has 2 kinds of conditions:

* dry crack / dry inclusion

$$U_1=\frac{16(\lambda+2\mu)}{3(3\lambda+4\mu)}$$

$$U_3=\frac{4(\lambda+2\mu)}{3(3\lambda+\mu)}$$

* "weak" inclusion



In [0]:
# criteria for weak or not

Effective stiffness moduli:

$$C_{ij}^*=\begin{bmatrix} C_{11}^* & C_{12}^* & C_{13}^* & 0 & 0 & 0 \\ C_{21}^* & C_{22}^* & C_{23}^* & 0 & 0 & 0 \\ C_{31}^* & C_{32}^* & C_{33}^* & 0 & 0 & 0 \\ 0 & 0 & 0 & C_{44}^* & 0 & 0 \\ 0 & 0 & 0 & 0 & C_{55}^* & 0 \\ 0 & 0 & 0 & 0 & 0 & C_{66}^* \end{bmatrix}$$

Where $$C_{ij}^*=C_{ij}^0+C_{ij}^1$$